In [2]:
import os
import keyboard
from dotenv import load_dotenv
import tempfile
import openai
import sounddevice as sd
import soundfile as sf
from elevenlabs import generate, play, set_api_key

load_dotenv()

# Set your API keys
openai.api_key = os.getenv("OPENAI_API_KEY")
set_api_key(os.getenv("ELEVENLABS_API_KEY"))

# Initialize messages outside the loop
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant. Answer in 100 words or less",
    }
]

# Initialize audio parameters
duration = 5
fs = 44100
channels = 1

# Initialize Eleven Labs voice and model
voice = "Drew"
model = "eleven_multilingual_v1"

while True:
    print("Press space to start")
    keyboard.wait("space")

    # Record audio
    print('Recording ...')
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=channels)
    sd.wait()
    print('Finished recording ...')

    # Transcribe audio
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio:
        sf.write(temp_audio.name, recording, fs)
        temp_audio.seek(0)

    file = open(temp_audio.name, "rb")
    transcript = openai.Audio.transcribe("whisper-1", file=file)
    messages.append({"role": "user", "content": transcript["text"]})
    file.close()
    os.remove(temp_audio.name)

    # Chat completion
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
    response_message = response["choices"][0]["message"]["content"]

    # Print and play response
    print(f"Chatbot: {response_message}")
    play(generate(response_message, voice=voice, model=model))

    if keyboard.is_pressed("esc"):
        break


Press space to start
Recording ...
Finished recording ...
Chatbot: The biggest dog breed in the world is the English Mastiff, known for its large size and powerful build. They can weigh up to 200 pounds or more and stand over 30 inches at the shoulder.


KeyboardInterrupt: 